# After refactoring...

In [ ]:
import signalflow as sf
from pixasonics.core import App, Mapper
from pixasonics.features import MeanPixelValue
from pixasonics.synths import Theremin

# Create app
app = App()

In [2]:
# load an image
img_path = "images/cellular_dataset/Timepoint_001_220518-ST_C03_s1.jpg"
img = app.load_image(img_path)

In [3]:
# Create objects
mean_pix = MeanPixelValue()
theremin = Theremin()
pix2freq = Mapper(mean_pix, theremin["frequency"], exponent=2, out_high=1000)

In [ ]:
app.features

In [ ]:
app.attach_feature(mean_pix)
app.features

In [ ]:
app.synths

In [ ]:
app.attach_synth(theremin)
app.synths

In [ ]:
app.mappers

In [ ]:
app.attach_mapper(pix2freq)
app.mappers

In [ ]:
app.detach_mapper(pix2freq)
app.mappers

In [ ]:
# test adding another mapping: mean pixel value to amplitude with an exponential curve
pix2amp = Mapper(mean_pix, theremin["amplitude"], exponent=2)
app.attach_mapper(pix2amp)

In [ ]:
pix2pan = Mapper(mean_pix, theremin["panning"])
app.attach_mapper(pix2pan)

In [ ]:
# detach a mapper
app.detach_mapper(pix2amp)
app.mappers

In [ ]:
# detach old frequency mapping and add a new one with different scaling
app.detach_mapper(pix2freq)
pix2freq = Mapper(mean_pix, theremin["frequency"], exponent=2, out_low=100, out_high=2000)
app.attach_mapper(pix2freq)
app.mappers

In [ ]:
graph = sf.AudioGraph.get_shared_graph()
print(graph.structure)
print(graph.status)

# Import widget stuff

In [16]:
from ipywidgets import ToggleButton, Button, ButtonStyle, Layout, jslink, IntText, IntSlider, FloatSlider, GridBox, GridspecLayout, Box, VBox, Label, Accordion

# Mapper card proto

In [ ]:
mapper_label = Label(value="Mapper", style=dict(
    font_weight='bold',
    font_size='2em'))
mapper_id = Label(value="# ID", style=dict(
    font_weight='bold',
    font_size='1em',
    text_color='gray'))

top_row = Box([mapper_label, mapper_id], layout=Layout(justify_content='space-between'))

from_label = Label(value="From:")
from_value = Label(value="MeanPixelValue")
from_row = Box([from_label, from_value], layout=Layout(justify_content='space-between'))

to_label = Label(value="To:")
to_value = Label(value="Theremin Frequency")
to_row = Box([to_label, to_value], layout=Layout(justify_content='space-between'))

detach_btn = Button(
    description="Detach", 
    button_style='danger', 
    icon='unlink',
    layout=Layout(max_width='80px'))
detach_btn.on_click(lambda x: print("Detaching mapper"))
detach_row = Box([detach_btn], layout=Layout(justify_content='flex-end'))

mapper_card = GridBox(children=[top_row, from_row, to_row, detach_row],
                        layout=Layout(
                            width='auto', 
                            grid_template_columns='auto', 
                            grid_template_rows='1fr 0.8fr 1.6fr 1fr',
                            max_width='260px',
                            min_height='140px',
                            border='1px solid black',
                            padding='5px',
                            margin='5px'))

mapper_card


# Feature card proto

In [ ]:
from ipywidgets import Label, Layout, Box, VBox, GridBox, Button, IntSlider, FloatSlider, ToggleButton, Accordion, Text
import numpy as np

class FeatureCard():
    def __init__(
            self,
            name: str = "Feature", 
            id: str = "# ID", 
            min: str = "(str(min array))",
            max: str = "(str(max array))",
            value: str = "(str(value array))",
    ):
        self.name = name
        self.id = id
        self.min = min
        self.max = max
        self.value = value
        self.app = None
        self.feature = None
        self.create_ui()

    def __call__(self):
        return self.card

    def detach_callback(self, b):
        print("FeatureCard: detaching feature", self.id)
        if self.app is not None and self.feature is not None:
            self.app.detach_feature(self.feature)

    def reset_callback(self, b):
        print("FeatureCard: resetting min max", self.id)
        if self.feature is not None:
            self.feature.reset_min_max()

    def create_ui(self):
        feature_label = Label(
            value=self.name, 
            style=dict(
                font_weight='bold',
                font_size='20px'))
        
        feature_id = Label(
            value=self.id, 
            style=dict(
                font_weight='bold',
                font_size='10px',
                text_color='gray'))

        top_block = Box(
            [feature_label, feature_id], 
            layout=Layout(
                justify_content='space-between',
                align_items='flex-start',
                flex_flow='row',
                width='100%'))

        min_label = Label(value="Min:")
        min_value = Text(
            value=self.min,
            placeholder='(min array)',
            description='',
            disabled=True,
            layout=Layout(width='80%')
        )
        min_value.tag = "min"
        min_block = Box(
            [min_label, min_value], 
            layout=Layout(
                justify_content='space-between',
                flex_flow='row',
                width='100%'))
        
        max_label = Label(value="Max:")
        max_value = Text(
            value=self.max,
            placeholder='(max array)',
            description='',
            disabled=True,
            layout=Layout(width='80%')
        )
        max_value.tag = f"max"
        max_block = Box(
            [max_label, max_value], 
            layout=Layout(
                justify_content='space-between',
                flex_flow='row',
                width='100%'))
        
        value_label = Label(value="Value:")
        value_value = Text(
            value=self.value,
            placeholder='(value array)',
            description='',
            disabled=True,
            layout=Layout(width='80%')
        )
        value_value.tag = f"value"
        value_block = Box(
            [value_label, value_value], 
            layout=Layout(
                justify_content='space-between',
                flex_flow='row',
                width='100%'))


        reset_btn = Button(
            description="Reset", 
            button_style='warning', 
            icon='refresh',
            layout=Layout(max_width='80px'))
        reset_btn.on_click(self.reset_callback)

        detach_btn = Button(
            description="Detach", 
            button_style='danger', 
            icon='unlink',
            layout=Layout(max_width='80px'))
        detach_btn.on_click(self.detach_callback)
        
        btn_row = Box(
            [reset_btn, detach_btn], 
            layout=Layout(
                width='100%',
                justify_content='space-between'))

        self.card = Box(
            children=[top_block, min_block, max_block, value_block, btn_row],
            layout=Layout(
                width='auto', 
                # grid_template_columns='auto', 
                # grid_template_rows='1fr, 1fr, 1fr 1fr 1fr 1fr',
                flex_flow='column',
                align_items='flex-start',
                justify_content='flex-start',
                max_width='260px',
                min_height='100px',
                border='1px solid black',
                padding='5px',
                margin='5px'))
        self.card.tag = f"feature_{self.id}"

test_min_array = str(np.random.rand(10))
print(test_min_array)
test_max_array = str(np.random.rand(10))
print(test_max_array)
test_value_array = str(np.random.rand(10))
print(test_value_array)
feature_card = FeatureCard("Mean Value", "mean_pix", test_min_array, test_max_array, test_value_array)
feature_card()

# Carousel proto

In [ ]:
# Create a vertical carousel of mapper cards
mapper_cards = [mapper_card for i in range(10)]
mapper_carousel = Box(children=mapper_cards, layout=Layout(overflow='scroll',
                                                            border='3px solid black',
                                                            padding='5px',
                                                            width='auto',
                                                            max_width='300px',
                                                            max_height='500px',
                                                            flex_flow='column',
                                                            display='flex'))
mapper_carousel

In [ ]:
# access the children of the carousel
mapper_carousel.children

# remove a card
mapper_carousel.children = mapper_carousel.children[:-1]

mapper_carousel

# App layout proto

In [ ]:
from ipywidgets import ToggleButton, Button, ButtonStyle, Layout, jslink, IntText, IntSlider, FloatSlider, GridBox, GridspecLayout, Box, VBox, Label, Accordion
from pixasonics.ui import AppUI, ProbeSettings, AudioSettings, Model, find_widget_by_tag

probe_settings = ProbeSettings()
audio_settings = AudioSettings()

app_ui = AppUI(
    probe_settings,
    audio_settings
    )

app_ui()

In [22]:
# add a new canvas
test_canvas = Button(description="Test Canvas", layout=Layout(width='500px', height='500px'))
app_ui.box.children[0].children = [test_canvas]

In [23]:
# add a new canvas the new way
test_canvas = Button(description="Test Canvas", layout=Layout(width='500px', height='500px'))
app_canvas = find_widget_by_tag(app_ui.box, "app_canvas")
app_canvas.children = [test_canvas]

In [24]:
# add buttons to the features carousel
features_carousel = app_ui.box.children[1].children[0].children[2]
features_carousel.children = list(features_carousel.children) + [Button(description=f"Feature {i}") for i in range(10)]

In [25]:
# add buttons the new way
test_buttons = [Button(description=f"Feature {i}") for i in range(10)]
features_carousel = find_widget_by_tag(app_ui.box, "features_carousel")
features_carousel.children = list(features_carousel.children) + test_buttons

In [26]:
# delete all buttons from the features carousel
features_carousel.children = []